In [ ]:
import requests as req 
from bs4 import BeautifulSoup as bs
import re
import json

In [ ]:
with open("API_KEY", "r") as file:
    API_KEY = file.read()

In [ ]:
links = [
    #1
    {"url": "https://dls.iran-gamecenter-host.com/DonyayeSerial/movie/YYYY/",   "type": "title"},
    {"url": "https://dls.iran-gamecenter-host.com/DonyayeSerial/movie3/YYYY/",  "type": "id"   },
    {"url": "https://dls.iran-gamecenter-host.com/DonyayeSerial/series/",       "type": "title"},
    {"url": "https://dls.iran-gamecenter-host.com/DonyayeSerial/series2/",      "type": "id"   },
    {"url": "https://dls.iran-gamecenter-host.com/DonyayeSerial/series3/",      "type": "id"   },
    #2
    {"url": "https://dls2.iran-gamecenter-host.com/DonyayeSerial/movie/YYYY/",  "type": "title"},
    {"url": "https://dls2.iran-gamecenter-host.com/DonyayeSerial/movies/YYYY/", "type": "id"   },
    {"url": "https://dls2.iran-gamecenter-host.com/DonyayeSerial/series/",      "type": "title"},
    {"url": "https://dls2.iran-gamecenter-host.com/DonyayeSerial/series2/",     "type": "id"   },
    {"url": "https://dls2.iran-gamecenter-host.com/DonyayeSerial/series3/",     "type": "id"   },
    #4
    {"url": "https://dls4.iran-gamecenter-host.com/DonyayeSerial/movies/",      "type": "id"   },
    {"url": "https://dls4.iran-gamecenter-host.com/DonyayeSerial/movie3/",      "type": "id"   },
    {"url": "https://dls4.iran-gamecenter-host.com/DonyayeSerial/series/",      "type": "title"},
    {"url": "https://dls4.iran-gamecenter-host.com/DonyayeSerial/series2/",     "type": "id"   },
    {"url": "https://dls4.iran-gamecenter-host.com/DonyayeSerial/series3/",     "type": "id"   },
    #5
    {"url": "https://dls5.iran-gamecenter-host.com/DonyayeSerial/series/",      "type": "title"},
    {"url": "https://dls5.iran-gamecenter-host.com/DonyayeSerial/series2/",     "type": "id"   },
    {"url": "https://dls5.iran-gamecenter-host.com/DonyayeSerial/series3/",     "type": "id"   },
    {"url": "https://dls5.iran-gamecenter-host.com/DonyayeSerial/movies/YYYY/", "type": "id"   },
    {"url": "https://dls5.iran-gamecenter-host.com/DonyayeSerial/movie3/YYYY/", "type": "id"   },
    #6
    {"url": "https://dls6.iran-gamecenter-host.com/DonyayeSerial/movie/YYYY/",  "type": "title"},
    {"url": "https://dls6.iran-gamecenter-host.com/DonyayeSerial/movie3/YYYY/", "type": "id"   },
    {"url": "https://dls6.iran-gamecenter-host.com/DonyayeSerial/movies/YYYY/", "type": "id"   },
    {"url": "https://dls6.iran-gamecenter-host.com/DonyayeSerial/series/",      "type": "title"},
    {"url": "https://dls6.iran-gamecenter-host.com/DonyayeSerial/series2/",     "type": "id"   },
    {"url": "https://dls6.iran-gamecenter-host.com/DonyayeSerial/series3/",     "type": "id"   },
    #7
    {"url": "https://dls7.iran-gamecenter-host.com/DonyayeSerial/movies/YYYY/", "type": "id"   },
    {"url": "https://dls7.iran-gamecenter-host.com/DonyayeSerial/movie3/YYYY/", "type": "id"   },
    {"url": "https://dls7.iran-gamecenter-host.com/DonyayeSerial/series2/",     "type": "id"   },
    {"url": "https://dls7.iran-gamecenter-host.com/DonyayeSerial/series3/",     "type": "id"   }
]

MIN_YEAR = 1921
MAX_YEAR = 2025

In [ ]:
def read_urls(url):
    try:
        r = req.get(url)
    except:
        print("[X] Connection failed\nFinish.")
        exit()
    soup = bs(r.text, "html.parser")

    file_items = soup.select("td.n > a")

    found_urls = []
    for item in file_items:
        href = item.get("href")
        if href != "../":
            found_urls.append(href)

    return found_urls

In [ ]:
def imdb_data(id):
    try:
            r = req.get(f"http://www.omdbapi.com/?i={id}&apikey={API_KEY}")
    except:
        print("[X] Connection failed\nFinish.")
        exit()
    return r.json()

imdb_recognized = {}

def get_name(string):
    if string.startswith("tt"):
        try:
            name = imdb_recognized[string]
        except:
            name = imdb_data(string)["Title"]
            imdb_recognized[string] = name
    else:
        name = string.replace(".", " ")
    return name

def get_title(url):
    return re.findall(r"((movie|series)[s|\d]?\/(\d\d\d\d\/)?(\w|\.)+)", url)[0][0].split("/")[-1]

In [ ]:
def get_quality(url):
    try:
        return " ".join(re.findall(r"(\.\d{3,4}p\.?.*(Sub|Dub))", url)[0][0].split(".")[1:-1])
    except:
        return "Unknown Quality"
    
def get_season(url):
    try:
        season = re.findall(r"S\d+E\d+", url)[0].split("E")[0].replace("S", "")
    except:
        season = 0
    return season

def get_episode(url):
    try:
        episode = re.findall(r"S\d+E\d+", url)[0].split("E")[1]
    except:
        episode = 0
    return episode

def sub_or_dubbed(url):
    return re.findall(r"(dub|sub)", url.lower())[0]

def series_or_movie(url):
    return re.findall(r"(series|movie)", url.lower())[0]

def get_datacenter_info(url):
    return "-".join(re.findall(r"(dls\d?).*((movie|serie)s?\d?)", url)[0][0:-1])

In [ ]:
films = {
    "movie": {},
    "series": {}
}

def create_name_if_not_exists(som, name):
    try:
        if films[som][name]:
            pass
    except:
        films[som][name] = {}
def create_sob_if_not_exists(som, name, sob):
    try:
        if films[som][name][sob]:
            pass
    except:
        films[som][name][sob] = {}
def create_quality_if_not_exists(som, name, sob, quality):
    try:
        if films[som][name][sob][quality]:
            pass
    except:
        films[som][name][sob][quality] = {}
def create_season_if_not_exists(som, name, sob, quality, season):
    try:
        if films[som][name][sob][quality][season]:
            pass
    except:
        films[som][name][sob][quality][season] = {}

def create_episode_if_not_exists(som, name, sob, quality, season, episode):
    try:
        if films[som][name][sob][quality][season][episode]:
            pass
    except:
        films[som][name][sob][quality][season][episode] = []

def create_all_if_not_exist(som, name, sob, quality, season, episode):
    create_name_if_not_exists(som, name)
    create_sob_if_not_exists(som, name, sob)
    create_quality_if_not_exists(som, name, sob, quality)
    create_season_if_not_exists(som, name, sob, quality, season)
    create_episode_if_not_exists(som, name, sob, quality, season, episode)    

def classify_url(url):
    som = series_or_movie(url) # som stands for series or movie
    name = get_name(get_title(url))
    sob = sub_or_dubbed(url)  # sob stands for sub or dubbed
    quality = get_quality(url)
    season = get_season(url)
    episode = get_episode(url)
    
    create_all_if_not_exist(som, name, sob, quality, season, episode)
    
    films[som][name][sob][quality][season][episode].append(url)

In [ ]:
def is_visited(url):
    try:
        return visited[url]
    except:
        return False

# DFS on Website!

counter = 0
visited = {}
extracted = []

def dfs(url):
    global counter
    counter += 1
    print(f"[{counter}] {url}")
    items = read_urls(url)
    visited[url] = True
    for item in items:
        if item.endswith((".mkv", "mp4", ".srt", ".mka", ".mp3")):
            extracted.append(url + item)
            continue
        new_url = url + item
        if (not is_visited(new_url)):
            dfs(new_url)

In [ ]:
def save_data(films, file_name):
    json_output = json.dumps(films)
    with open(file_name, "w") as file:
        file.write(json_output)

In [ ]:
link_counter = 0
for link in links:
    link_counter += 1
    print(f"Scrapping {link["url"]} ({link_counter} of {len(links)})")
    if "YYYY" in link["url"]:
        for i in range(MIN_YEAR, MAX_YEAR+1):
            dfs(link["url"].replace("YYYY", str(i)))
    dfs(link["url"])
    save_data(extracted, get_datacenter_info(link["url"]) + ".json")
    extracted = []
    visited = {}
    counter = 0
print("Scrapping Done.")

In [ ]:
# matching json files
counter = 0
for link in links:
    counter += 1
    print(f"{counter} of {len(links)}")
    file_name = get_datacenter_info(link["url"])
    with open(file_name + ".json", "r") as file:
        saved_data = json.loads(file.read())
        for url in saved_data:
            classify_url(url)
print("Done.")
save_data(films, "final.json")
print("File saved.")